# Age of Empires 2 game records analysis

Importing modules needed

In [1]:
import sys
import glob, os
from datetime import datetime as dt
from mgz import header, body
from mgz.summary import Summary
import json
from mgz.model import parse_match, serialize
import json
import re
import pandas as pd
from functools import reduce
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn 
import math

os.chdir("C:\\Users\\kardo\\Games\\Age of Empires 2 DE\\76561198861901568\\savegame")


Importing files to be parsed, those are personal game replay files (381 of them at the time of this writing) and involve Multiplayer (online) 1 versus 1 games

In [2]:
list_of_files = glob.glob("*.aoe2record")
len(list_of_files)
list_of_files[-5:] # showing last five records

['MP Replay v101.102.8319.0 #(73855) @2022.12.08 104819 (2).aoe2record',
 'MP Replay v101.102.8319.0 #(73855) @2022.12.08 114046 (1).aoe2record',
 'MP Replay v101.102.8319.0 #(73855) @2022.12.08 120821 (1).aoe2record',
 'MP Replay v101.102.8319.0 #(73855) @2022.12.08 124146 (2).aoe2record',
 'MP Replay v101.102.8319.0 #(73855) @2022.12.08 125748 (2).aoe2record']

So here there are a varierty of headers (from json) within which things ranging from 'research' (a given tech was researched), to 'move' - in this case included in the 'actions' key/header. 
The main focus in this next section will be on focusing on 'follow' keystroke which involves a scout (to which both players have access at the very start of the game) to lure deer into the Town Center/starting base for additional 'free' source of food.

In [3]:
matches = []
json_obj = []

for file in list_of_files:
    try: 
        with open(str(file), 'rb') as rec:
            matches.append(json.dumps(serialize(parse_match(rec)))) 
    except Exception:
                pass

for dumps in matches:
    json_obj.append(json.loads(dumps))

len(json_obj)
[len(x) for x in matches]


# player actions 
def actions_taken(json_obj):
	actions_taken = json_obj['actions']
	str_conv = str(actions_taken)
	dict_conv = eval(str_conv)
	return dict_conv

# players identification
def players_(json_obj):
	players = json_obj['players']
	str_conv = str(players)
	dict_conv = eval(str_conv)
	return dict_conv

actions = []
for game_files in json_obj:
    actions.append(actions_taken(game_files))

players = []
for game_files in json_obj:
    players.append(players_(game_files))

# generating multiple dataframes from json to get cleaner values and later run analysis
def from_json_to_df(x):

    df = []
    for games_played in x:
        df.append(pd.json_normalize(games_played)) 
    return df

dfs_with_player_mappings = []

for left, right in zip(from_json_to_df(actions), from_json_to_df(players)):
      dfs_with_player_mappings.append( pd.merge(left, right, how = 'left', left_on = 'player', right_on = 'number') )


deer_pushed_cnt = []
won_game_cnt = []
eapm = []
wessex_civ = []

def counting_intentional_deer_pushes(x):

    ttl_deer_pushed = games['payload.target_id'].loc[( games['type'] == 'FOLLOW' )\
             & ( games['name'] == 'AofWessex' )].groupby(games['payload.target_id']).count()
    mask = ttl_deer_pushed > 3        
    return len(ttl_deer_pushed.loc[mask])

for games in dfs_with_player_mappings:

    try:
        deer_pushed_cnt.append(counting_intentional_deer_pushes(games))
    except Exception:
                pass

    won_game_cnt.append(games['winner'].loc[( games['winner'] == True )\
         & ( games['name'] == 'AofWessex' )].drop_duplicates().count())

    eapm.append(games['eapm'].loc[( games['name'] == 'AofWessex' )].drop_duplicates().min())
    
    wessex_civ.append(games['civilization'].loc[( games['name'] == 'AofWessex' )].drop_duplicates().min())
    

overall_stats_df = pd.DataFrame(list(zip(deer_pushed_cnt, eapm, wessex_civ, won_game_cnt)),\
    columns = ['deer_pushed_ttl', 'eapm', 'civ', 'won_game'])


In [4]:
overall_stats_df

,deer_pushed_ttl,eapm,civ,won_game
0,0,42.0,Incas,0
1,0,45.0,Incas,0
2,0,45.0,Burmese,1
3,0,55.0,Burmese,1
4,0,46.0,Burmese,0
...,...,...,...,...
360,0,41.0,Vikings,0
361,1,50.0,Portuguese,0
362,1,53.0,Britons,1
363,0,63.0,Burmese,1
